In [48]:
import os 
import pandas as pd


In [53]:
vector_data=pd.read_csv('D:\Model_Deployement\End_to_End_Phishing_mail_detection_ML_project\\artifacts\data_preprocessed\\vectorized_data.csv',index_col=False)

In [54]:
k=vector_data['Email_vector']

In [55]:
import numpy as np
def convert_string_to_array(vector_string):
    """Convert string representation of vector to numpy array"""
    try:
        # Remove brackets and split by whitespace
        vector_string = vector_string.strip('[]')
        # Convert to numpy array
        return np.fromstring(vector_string, sep=' ')
    except:
        return None

vector_data['Email_vector'] = vector_data['Email_vector'].apply(convert_string_to_array)

    
 # Example usage, convert the first vector string to numpy array

In [61]:
vector_data[vector_data.columns[2:]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18650 entries, 0 to 18649
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Email Text    18634 non-null  object
 1   Email Type    18650 non-null  int64 
 2   tokens        18650 non-null  object
 3   Email_vector  18650 non-null  object
dtypes: int64(1), object(3)
memory usage: 582.9+ KB


In [23]:
vec=np.vstack(new)

In [25]:
vec.shape

(100, 1)

In [62]:
vector_data['Email Type'].replace({'Safe Email':0,'Phishing Email':1},inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_12088\2393261195.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  vector_data['Email Type'].replace({'Safe Email':0,'Phishing Email':1},inplace=True)


In [63]:
vector_data['Email Type']

0        0
1        0
2        0
3        1
4        1
        ..
18645    1
18646    0
18647    0
18648    0
18649    1
Name: Email Type, Length: 18650, dtype: int64

In [65]:
vector_data['Email Type'].unique()

array([0, 1], dtype=int64)

# Model training on the data

In [1]:
import os
from pathlib import Path


In [4]:
os.getcwd()

'd:\\Model_Deployement\\End_to_End_Phishing_mail_detection_ML_project'

In [3]:
os.chdir('../')

In [25]:
from dataclasses import dataclass
@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir :Path
    training_data_path: Path
    trained_model_dir: Path


In [26]:
from Phising_mail_detection.constants import *
from Phising_mail_detection.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_preprocessing_config(self) -> ModelTrainingConfig:
        config = self.config.Model_training
        # schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        Model_train_config = ModelTrainingConfig(
            root_dir = config.root_dir,
            training_data_path= config.training_data_path,
            trained_model_dir= config.trained_model_dir
        )

        return Model_train_config


In [33]:
from Phising_mail_detection import logger
import numpy as np
from Phising_mail_detection.utils.common import load_model,save_model
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,classification_report
from lightgbm import LGBMClassifier


class Model_training_function:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config
    
    def convert_string_to_array(self,vector_string):
        
      """Convert string representation of vector to numpy array"""
      try:
        # Remove brackets and split by whitespace
            vector_string = vector_string.strip('[]')
        # Convert to numpy array
            return np.fromstring(vector_string, sep=' ')
      except Exception as e:
            return e
      

    def train_model(self):
        logger.info("Loading training data")
        training_data = pd.read_csv(self.config.training_data_path)
        training_data['Email_vector'] = training_data['Email_vector'].apply(self.convert_string_to_array)

        logger.info("Splitting data into features and target")
        X = training_data['Email_vector']
        y = training_data['Email Type'].values
        logger.info("Splitting data into train and test sets")
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        X_train = np.vstack(X_train)
        X_test= np.vstack(X_test)
        print(X_train.shape)
        logger.info("Training the model")
        # model = LGBMClassifier()
        # model = LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
        model=RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)
        logger.info("Model evaluation on training data")
        y_train_pred = model.predict(X_train)
        train_accuracy = accuracy_score(y_train, y_train_pred)
        train_classification_rep = classification_report(y_train, y_train_pred)
        print(f"Training Accuracy: {train_accuracy}")
        print("Training Classification Report:")
        print(train_classification_rep)
        logger.info("Evaluating the model")
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        classification_rep = classification_report(y_test, y_pred)
        print(f"Accuracy: {accuracy}")
        print("Classification Report:")
        print(classification_rep)
        logger.info("Saving the trained model")
        save_model(model, self.config.trained_model_dir)


    
        


    


In [34]:
try:
    config = ConfigurationManager()
    model_training_config = config.get_data_preprocessing_config()
    model= Model_training_function(config=model_training_config)
    model.train_model()
except Exception as e:
    logger.exception(e)
    raise e

[2025-08-05 00:27:15,926:'INFO':common:yaml file: config\config.yaml loaded successfully]
[2025-08-05 00:27:15,931:'INFO':common:yaml file: params.yaml loaded successfully]
[2025-08-05 00:27:15,933:'INFO':common:yaml file: schema.yaml loaded successfully]
[2025-08-05 00:27:15,934:'INFO':common:created directory at: artifacts]
[2025-08-05 00:27:15,936:'INFO':common:created directory at: artifacts/model_training]
[2025-08-05 00:27:15,938:'INFO':14188678:Loading training data]
[2025-08-05 00:27:18,166:'INFO':14188678:Splitting data into features and target]
[2025-08-05 00:27:18,166:'INFO':14188678:Splitting data into train and test sets]
(14920, 100)
[2025-08-05 00:27:18,220:'INFO':14188678:Training the model]
[2025-08-05 00:27:38,026:'INFO':14188678:Model evaluation on training data]
Training Accuracy: 0.988941018766756
Training Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      9049
           1       0.97      